In [20]:
from bs4 import BeautifulSoup

#The request like an actual individual actually makes a real "request" to a site for information.
import requests

# Web scrapping from an external site, using its url as provided in within the argument of the get function
html_text = requests.get('https://m.timesjobs.com/mobile/jobs-search-result.html?txtKeywords=Python%2C&cboWorkExp1=-1&txtLocation=').text
soup = BeautifulSoup(html_text, 'lxml')
job = soup.find('li', class_ = 'ui-content search-result')
print(job)

None


In [46]:
from bs4 import BeautifulSoup
import requests
import csv

# Information related to the site you want to scrape.
page_to_scrape = requests.get("https://quotes.toscrape.com")
soup = BeautifulSoup(page_to_scrape.text, "html.parser")
                     
# The first argument of the findAll method is the title of the tag while the second argument is the attribute name and its value e.g class: text.
quotes = soup.findAll("span", attrs={"class": "text"})
authors = soup.findAll("small", attrs={"class": "author"})

# Writing the scrapped data into a CSV file to be called "scrapped_quote.csv"
file = open("scraped_quote.csv", "w")
writer = csv.writer(file)

# Using a for loop to iterate through all the quote.
for quote, author in zip(quotes, authors):
    print(quote.text + " , " + author.text)
    # writting the iterated content into the csv file
    writer.writerow([quote.text, author.text])

file.close()

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” , Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.” , J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” , Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” , Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” , Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.” , Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.” , André Gide
“I have not failed. I've just found 10,000 ways that won't work.” , Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot water.” , Eleanor Roos

In [69]:
from bs4 import BeautifulSoup
import requests
import csv

# Information related to the site you want to scrape.
page_to_scrape = requests.get("https://www.vanguardngr.com/category/sports/")
soup = BeautifulSoup(page_to_scrape.text, "html.parser")
                     
# The first argument of the findAll method is the title of the tag while the second argument is the attribute name and its value e.g class: text.
head_lines = soup.findAll("h3", attrs={"class": "entry-title"})
#contents = soup.findAll("p")
contents = soup.findAll("div", attrs={"class": "entry-excerpt"})

# Writing the scrapped data into a CSV file to be called "scrapped_quote.csv"
file = open("scraped_news.csv", "w")
writer = csv.writer(file)

# Using a for loop to iterate through all the quote.
for head_line, content in zip(head_lines, contents):
    print(head_line.text + " , " + content.text)
    #writting the iterated content into the csv file
    writer.writerow([head_line.text, content.text])

file.close()


PSG beat Arsenal 3-1 on aggregate to reach Champions League final
 , Inter Milan come back down to earth on Sunday when they travel to Torino, hoping to relaunch their Serie A title defence, still buzzing from one of the greatest nights of European football in their history.

Tottenham star James Maddison to miss rest of season
 , 
Paris Saint-Germain forward Ousmane Dembele, who suffered a hamstring injury last week, has been passed fit to play in their Champions League semi-final second leg against Arsenal, coach Luis Enrique said on Tuesday.


Inter turn attention to fading Serie A title defence after Barca triumph
 , 
Ousmane Dembele and his fellow attacking players may have stolen most of the limelight on Paris Saint-Germain’s run in this season’s Champions League, but the form of goalkeeper Gianluigi Donnarumma has been just as important in taking the French giants to the brink of the final.


Dembele starts on bench for PSG against Arsenal in UCL
 , 
In an expected turn of even

In [70]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

base_url = "https://www.vanguardngr.com/category/sports"
headers = {"User-Agent": "Mozilla/5.0"}

all_data = []
page_num = 1

while len(all_data) < 100:
    print(f"Scraping page {page_num}...")
    url = f"{base_url}/page/{page_num}/"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Get all article links on the page
    articles = soup.find_all("h3", class_="entry-title")
    links = [a.find("a")["href"] for a in articles if a.find("a")]

    for link in links:
        try:
            article_page = requests.get(link, headers=headers)
            article_soup = BeautifulSoup(article_page.text, "html.parser")
            content_div = article_soup.find("div", class_="entry-content-inner-wrapper")
            if not content_div:
                continue
            paragraphs = content_div.find_all("p")
            for p in paragraphs:
                raw_text = p.get_text(strip=True)
                if len(raw_text) > 40:  # filter out short junk
                    cleaned = clean_text(raw_text)
                    all_data.append([raw_text, cleaned, "Sports", link])
                    if len(all_data) >= 100:
                        break
            time.sleep(1)  # polite delay
        except Exception as e:
            print(f"Error processing {link}: {e}")
    page_num += 1

# Save to CSV
with open("vanguard_sports_paragraphs.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Text", "Cleaned_Text", "Category", "Source"])
    writer.writerows(all_data)

print(f"\n✅ Done. Saved {len(all_data)} paragraphs to vanguard_sports_paragraphs.csv.")


ModuleNotFoundError: No module named 'nltk'

In [71]:
import subprocess
import sys

# Try to import nltk, install if not found
try:
    import nltk
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk"])
    import nltk

import requests
from bs4 import BeautifulSoup
import csv
import re
import time

# Download stopwords if not already downloaded
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    text = text.lower()  # Lowercase
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text

base_url = "https://www.vanguardngr.com/category/sports"
headers = {"User-Agent": "Mozilla/5.0"}

all_data = []
page_num = 1

while len(all_data) < 100:
    print(f"Scraping page {page_num}...")
    url = f"{base_url}/page/{page_num}/"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    articles = soup.find_all("h3", class_="entry-title")
    links = [a.find("a")["href"] for a in articles if a.find("a")]

    for link in links:
        try:
            article_page = requests.get(link, headers=headers)
            article_soup = BeautifulSoup(article_page.text, "html.parser")
            content_div = article_soup.find("div", class_="entry-content-inner-wrapper")
            if not content_div:
                continue
            paragraphs = content_div.find_all("p")
            for p in paragraphs:
                raw_text = p.get_text(strip=True)
                if len(raw_text) > 40:
                    cleaned = clean_text(raw_text)
                    all_data.append([raw_text, cleaned, "Sports", link])
                    if len(all_data) >= 100:
                        break
            time.sleep(1)
        except Exception as e:
            print(f"Error processing {link}: {e}")
    page_num += 1

# Save to CSV
with open("vanguard_sports_paragraphs.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Text", "Cleaned_Text", "Category", "Source"])
    writer.writerows(all_data)

print(f"\n✅ Done. Saved {len(all_data)} paragraphs to vanguard_sports_paragraphs.csv.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Scraping page 1...

✅ Done. Saved 117 paragraphs to vanguard_sports_paragraphs.csv.
